# 12-семинар: K-Means, Иерархиялық кластерлеу және DBSCAN тәжірибеде

**Семинардың мақсаты:** Қарапайым синтетикалық деректерде үш негізгі кластерлеу алгоритмінің қалай жұмыс істейтінін, қай жерде қателесетінін және қалай бапталатынын кезең-кезеңімен талдау. Біз сондай-ақ алынған кластерлерді интерпретациялауды үйренеміз.

In [ ]:
import os
# Windows жүйесінде KMeans жадының жылыстау мәселесін шешу
os.environ['OMP_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors

sns.set_theme(style="whitegrid")

## 1-бөлім: K-Means — центроидтарға негізделген кластерлеу

**Деректер жинағы:** `make_blobs` — K-Means үшін өте қолайлы генерацияланған деректер (нүктелердің үш бөлек "бұлты").

In [ ]:
X_blobs, y_blobs = make_blobs(n_samples=500, centers=3, random_state=42, cluster_std=1.0)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_blobs[:,0], y=X_blobs[:,1])
plt.title("Бастапқы деректер (Blobs)")
plt.show()

**Түсініктеме:** *K-Means (және қашықтыққа негізделген кез келген басқа алгоритмді) қолданбас бұрын, деректерді **масштабтау өте маңызды**. Әйтпесе, мәндерінің диапазоны үлкен белгілер нәтижеге пропорционалды емес қатты әсер етеді.*

In [ ]:
scaler = StandardScaler()
X_blobs_scaled = scaler.fit_transform(X_blobs)

### 1.1. Шынтақ әдісі арқылы оңтайлы K-ны іздеу

**Түсініктеме:** *Біз деректерде қанша кластер бар екенін білмейміз. K-Means-ті әртүрлі `k` мәнімен циклде орындап, `inertia_` (WCSS) мәнін жазып отырамыз. Содан кейін график салып, "шынтақты" табамыз.*

In [ ]:
ssd = []
k_range = range(1, 11)

for k in k_range:
    model = KMeans(n_clusters=k, random_state=42, n_init='auto')
    model.fit(X_blobs_scaled)
    ssd.append(model.inertia_)

plt.figure(figsize=(8,6))
plt.plot(k_range, ssd, 'o-')
plt.xlabel("Кластерлер саны (K)")
plt.ylabel("Квадраттық қашықтықтар қосындысы (WCSS)")
plt.title("K-Means үшін шынтақ әдісі")
plt.xticks(k_range)
plt.show()

**Қорытынды:** *Графикте K=3 нүктесінде айқын "шынтақ" көрініп тұр. Бұл кластерлердің оңтайлы саны — үш екенін білдіреді.*

### 1.2. K-Means-тің соңғы моделі және интерпретация

**Түсініктеме:** *Модельді `k=3` деп оқытып, нәтижені визуализациялаймыз. Содан кейін алынған кластерлерді қалай интерпретациялауға болатынын көреміз.*

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
labels_kmeans = kmeans.fit_predict(X_blobs_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_blobs[:,0], y=X_blobs[:,1], hue=labels_kmeans, palette='viridis')
plt.title("K-Means нәтижесі (K=3)")
plt.show()

#### Кластерлерді интерпретациялау
**Түсініктеме:** *Алгоритм бізге 0, 1, 2 белгілерін қайтарды. Бұл сандардың өздігінен еш мағынасы жоқ. Талдаушылар ретінде біздің міндетіміз — оларға мағына беру. Негізгі тәсіл — бастапқы деректерді кластер белгілері бойынша топтап, белгілердің орташа мәндеріне қарау.*

In [ ]:
df_blobs = pd.DataFrame(X_blobs, columns=['Feature_1', 'Feature_2'])
df_blobs['Cluster'] = labels_kmeans

# Топтап, орташа мәндерді есептейміз
cluster_profile = df_blobs.groupby('Cluster').mean()
print(cluster_profile)
plt.figure(figsize=(10, 4))
sns.heatmap(cluster_profile, annot=True, cmap='viridis')
plt.title('K-Means кластерлерінің профилі')
plt.show()

**Қорытынды:** *Біз мынаны көріп отырмыз:
- **0-кластер** екі белгінің де жоғары мәндерімен сипатталады.
- **1-кластерде** Feature_1-дің мәні төмен, ал Feature_2-нің мәні жоғары.
- **2-кластерде** Feature_2-нің мәні төмен.
Осылайша біз әрбір кластердің "профилін" жасаймыз, бұл оларға мағыналы атаулар беруге мүмкіндік береді.*

## 2-бөлім: Иерархиялық кластерлеу

**Түсініктеме:** *Дендрограмманы көрнекі түрде салу үшін шағын, бірақ түсінікті деректер жинағын қолданамыз.*

In [ ]:
# Көрнекі деректер жинағын құрамыз (дәрістегідей)
X_hier = np.array([
    [8, 150], [9, 160], [8.5, 140], [7.5, 155], # Кеңсе қызметкерлері
    [3, 40], [4, 50], [2.5, 35], [3.5, 45],    # Студенттер
    [5, 200], [6, 210], [4.5, 190], [5.5, 220] # Фрилансерлер
])

# Дендрограмманы саламыз
linked = linkage(X_hier, method='ward')

plt.figure(figsize=(12, 8))
dendrogram(linked, orientation='top', labels=[f"Нүкте {i}" for i in range(len(X_hier))], distance_sort='descending')
plt.title('Иерархиялық кластерлеу дендрограммасы')
plt.ylabel('Уорд бойынша қашықтық')
plt.axhline(y=100, c='k', linestyle='--') 
plt.show()

**Қорытынды:** *Дендрограмма нүктелердің бірігу иерархиясын көрнекі түрде көрсетеді. `y=100` деңгейіндегі көлденең кесу 3 тік сызықты қиып өтеді, бұл 3 кластердің бар екендігі туралы гипотезамызды растайды.*

## 3-бөлім: DBSCAN — тығыздыққа негізделген кластерлеу

**Түсініктеме:** *Енді K-Means дәрменсіз болатын деректерге көшейік.*

In [ ]:
X_moons, y_moons = make_moons(n_samples=500, noise=0.1, random_state=42)
X_moons_scaled = StandardScaler().fit_transform(X_moons)

### 3.1. K-Means пен DBSCAN-ды салыстыру

**Түсініктеме:** *Екі алгоритмді де `moons` деректеріне қолданып, нәтижесін көрейік.*

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
dbscan = DBSCAN(eps=0.3)

labels_kmeans = kmeans.fit_predict(X_moons_scaled)
labels_dbscan = dbscan.fit_predict(X_moons_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.scatterplot(ax=axes[0], x=X_moons[:,0], y=X_moons[:,1], hue=labels_kmeans, palette='viridis')
axes[0].set_title('K-Means-тің сәтсіз нәтижесі')

sns.scatterplot(ax=axes[1], x=X_moons[:,0], y=X_moons[:,1], hue=labels_dbscan, palette='viridis')
axes[1].set_title('DBSCAN-ның сәтті нәтижесі')
plt.show()

### 3.2. DBSCAN үшін гиперпараметрлерді таңдау

**Түсініктеме:** *Біз `eps=0.3` мәнін қалай таңдадық? Бұл сиқыр емес. `eps`-ті таңдауға арналған пайдалы эвристика бар. Идеясы — k-шы ең жақын көршіге дейінгі қашықтықтар графигінде "иілу нүктесін" табу.*

1.  `min_samples`-ты таңдаймыз. Жақсы эмпирикалық ереже: `min_samples = 2 * белгілер_саны`. Бізде 2 белгі бар, сондықтан `min_samples=4` деп аламыз.
2.  Деректер жинағындағы **әрбір** нүкте үшін оның 4-ші ең жақын көршісіне дейінгі қашықтықты табамыз.
3.  Бұл қашықтықтарды өсу ретімен сұрыптап, график саламыз.
4.  Графиктен "шынтақты" іздейміз — қисықтың кенеттен жоғары қарай ұмтылатын нүктесі. `eps`-тің оңтайлы мәні шамамен осы деңгейде болады.

In [ ]:
min_samples = 2 * X_moons_scaled.shape[1]

neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X_moons_scaled)
distances, indices = neighbors_fit.kneighbors(X_moons_scaled)

# k-шы көршіге дейінгі қашықтықтарды сұрыптаймыз (k = min_samples)
k_distances = np.sort(distances[:, min_samples-1], axis=0)

plt.figure(figsize=(8,6))
plt.plot(k_distances)
plt.title('Epsilon-ды таңдауға арналған k-қашықтықтар графигі')
plt.xlabel("Қашықтық бойынша сұрыпталған нүктелер")
plt.ylabel(f"{min_samples}-ші көршіге дейінгі қашықтық")
plt.axhline(y=0.3, c='r', linestyle='--') # Біз болжаған шынтақ
plt.grid(True)
plt.show()

**Семинардың қорытындысы:** *Біз "ең жақсы" алгоритм жоқ екенін көрдік. K-Means жылдам және сфералық кластерлер үшін жақсы. Иерархиялық кластерлеу көрнекі дендрограмма береді. Тығыздыққа негізделген DBSCAN күрделі пішіндегі кластерлермен және шығарындылармен тамаша жұмыс істейді, бірақ гиперпараметрлерді мұқият баптауды қажет етеді.*